In [5]:
#查看导包路径
import sys 
sys.path

['/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python36.zip',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/lib-dynload',
 '',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/site-packages',
 '/home/xuenhui/anaconda3/envs/patch_base-py362/lib/python3.6/site-packages/IPython/extensions',
 '/home/xuenhui/.ipython',
 '/home/xuenhui/anaconda3/envs/patch-py362/lib/python36.zip',
 '/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6',
 '/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/lib-dynload',
 '/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/site-packages']

In [6]:
#添加导包路径
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python36.zip')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/lib-dynload')
sys.path.append('/home/xuenhui/anaconda3/envs/patch-py362/lib/python3.6/site-packages')

Solution: The 340-dimensional feature vector was selected using the baseline scheme and combined with the 5 name features extracted from the test1 scheme to form a final 345-dimensional vector, which was modeled using XGB, and the results after grid search were fused with those of the baseline method (using fea340). The highest scoring test3 was obtained (0.5911, 0.6656).

In [7]:
import os
import scipy.stats
import scipy.io
import numpy as np

def read_mat(mat_path):
  mat = scipy.io.loadmat(mat_path)
  mat = np.asarray(mat['feats_mat'], dtype=np.float)
  mat[np.isnan(mat)] = 0
  mat[np.isinf(mat)] = 0
  return mat
# load fea340
train_fea = read_mat(os.path.join('feats/selected_340', 'DFGC-train_DFGC1st_withstd_feats340.mat'))
test1_fea = read_mat(os.path.join('feats/selected_340', 'DFGC-test1_DFGC1st_withstd_feats340.mat'))
test2_fea = read_mat(os.path.join('feats/selected_340', 'DFGC-test2_DFGC1st_withstd_feats340.mat'))
test3_fea = read_mat(os.path.join('feats/selected_340', 'DFGC-test3_DFGC1st_withstd_feats340.mat'))


In [8]:
# Get the training set label
import pandas as pd
label_path = r'./data/label'
df_train = pd.read_csv(os.path.join(label_path, 'train_set.csv'), skiprows=[])
train_label = np.array(list(df_train['mos']), dtype=np.float)


In [9]:
train_fea.shape

(700, 340)

In [10]:
# load name_fea
import pandas as pd
import numpy as np
train = pd.read_csv('data/label/train_set.csv')
test1 = pd.read_csv('data/label/test_set1.txt', names=['file'])
test2 = pd.read_csv('data/label/test_set2.txt', names=['file'])
test3 = pd.read_csv('data/label/test_set3.txt', names=['file'])

In [11]:
features = train.drop(['mos'],axis = 1)

In [12]:
features['file_name'] = features['file'].apply(lambda x: x.split("/")[0])
features['id1'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
features['id2'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
features['id3'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
features['man'] = features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

test1_features = test1
test1_features['file_name'] = test1_features['file'].apply(lambda x: x.split("/")[0])
test1_features['id1'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test1_features['id2'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test1_features['id3'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test1_features['man'] = test1_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

test2_features = test2
test2_features['file_name'] = test2_features['file'].apply(lambda x: x.split("/")[0])
test2_features['id1'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test2_features['id2'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test2_features['id3'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test2_features['man'] = test2_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

test3_features = test3
test3_features['file_name'] = test3_features['file'].apply(lambda x: x.split("/")[0])
test3_features['id1'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[0])
test3_features['id2'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[1])
test3_features['id3'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[2])
test3_features['man'] = test3_features['file'].apply(lambda x: x.split("/")[1].split("-")[-1].split(".")[0])

In [13]:
# Make a tag before merging to facilitate splitting
features['tag'] = 'train'
test1_features['tag'] = 'test1'
test2_features['tag'] = 'test2'
test3_features['tag'] = 'test3'


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
# It is necessary to combine the training set and the test set, and then do LabelEncoder uniformly
all_df = pd.concat([features, test1_features, test2_features, test3_features])

In [14]:
all_df = all_df.drop(['file'],axis = 1)

In [15]:
# Perform LabelEncoder on id1
all_df['id1'] = all_df['id1'].astype('str')
all_df['id1'] = le.fit_transform(all_df['id1'])

all_df['id2'] = all_df['id2'].astype('str')
all_df['id2'] = le.fit_transform(all_df['id2'])

all_df['id3'] = all_df['id3'].astype('str')
all_df['id3'] = le.fit_transform(all_df['id3'])

all_df['file_name'] = all_df['file_name'].astype('str')
all_df['file_name'] = le.fit_transform(all_df['file_name'])

all_df['man'] = all_df['man'].astype('str')
all_df['man'] = le.fit_transform(all_df['man'])


In [16]:
# Split the encoded features into train, test1, test2, test3
features['file_name'] = all_df[all_df['tag']=='train']['file_name']
features['id1'] = all_df[all_df['tag']=='train']['id1']
features['id2'] = all_df[all_df['tag']=='train']['id2']
features['id3'] = all_df[all_df['tag']=='train']['id3']
features['man'] = all_df[all_df['tag']=='train']['man']
features = features.drop(['tag'],axis = 1)
features = features.drop(['file'],axis = 1)


test1_features['file_name'] = all_df[all_df['tag']=='test1']['file_name']
test1_features['id1'] = all_df[all_df['tag']=='test1']['id1']
test1_features['id2'] = all_df[all_df['tag']=='test1']['id2']
test1_features['id3'] = all_df[all_df['tag']=='test1']['id3']
test1_features['man'] = all_df[all_df['tag']=='test1']['man']
test1_features = test1_features.drop(['tag'],axis = 1)
test1_features = test1_features.drop(['file'],axis = 1)


test2_features['file_name'] = all_df[all_df['tag']=='test2']['file_name']
test2_features['id1'] = all_df[all_df['tag']=='test2']['id1']
test2_features['id2'] = all_df[all_df['tag']=='test2']['id2']
test2_features['id3'] = all_df[all_df['tag']=='test2']['id3']
test2_features['man'] = all_df[all_df['tag']=='test2']['man']
test2_features = test2_features.drop(['tag'],axis = 1)
test2_features = test2_features.drop(['file'],axis = 1)


test3_features['file_name'] = all_df[all_df['tag']=='test3']['file_name']
test3_features['id1'] = all_df[all_df['tag']=='test3']['id1']
test3_features['id2'] = all_df[all_df['tag']=='test3']['id2']
test3_features['id3'] = all_df[all_df['tag']=='test3']['id3']
test3_features['man'] = all_df[all_df['tag']=='test3']['man']
test3_features = test3_features.drop(['tag'],axis = 1)
test3_features = test3_features.drop(['file'],axis = 1)

In [17]:
# splicing of fea340 and name_fea

#convert data type：pandas.core.frame.DataFrame -->  numpy.ndarray
features = features.values
print(train_fea.shape)
print(features.shape)
print(type(train_fea))
print(type(features))

(700, 340)
(700, 5)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [18]:
train_fea

array([[-0.00405909, -2.01999879,  1.96036637, ...,  0.0287498 ,
         0.01503708,  0.03093584],
       [ 0.53068542, -0.5901233 ,  2.67612171, ...,  0.02590029,
         0.02505589,  0.02585017],
       [ 0.66390783, -1.39681458,  1.18705714, ...,  0.01451662,
         0.01090694,  0.02874019],
       ...,
       [-0.43294653, -1.75199115,  0.04795574, ...,  0.05205392,
         0.04071713,  0.05904517],
       [-0.16586778, -1.45879996,  0.29791114, ...,  0.05937014,
         0.02364715,  0.04718614],
       [ 0.14763498, -1.10856962,  0.50533921, ...,  0.05936996,
         0.02414202,  0.0443061 ]])

In [19]:
features

array([[ 0,  0,  0,  7,  0],
       [ 0,  0,  0,  7,  2],
       [ 0,  0,  0,  7,  3],
       ...,
       [ 2, 19,  1, 12, 30],
       [ 2, 19,  1, 12, 31],
       [ 2, 19,  1, 12, 32]])

In [20]:
train_fea = np.concatenate((train_fea,features),axis=1)

In [21]:
print(train_fea.shape)

(700, 345)


In [22]:
test1_fea = np.concatenate((test1_fea,test1_features),axis=1)
test1_fea.shape

(300, 345)

In [23]:
test2_fea = np.concatenate((test2_fea,test2_features),axis=1)
test3_fea = np.concatenate((test3_fea,test3_features),axis=1)

In [24]:
print(type(train_fea))

print(type(train_label))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [25]:
# Modeling with XGBRegressor
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(train_fea,train_label, test_size=0.3, random_state=1)


In [26]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=5,
         subsample=0.6, colsample_bytree=0.8, learning_rate=0.05, random_state=0)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.5930199207844162

In [27]:
import numpy as np
from sklearn.metrics import confusion_matrix, mean_squared_error
pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, pred))
rmse

0.5204358328825381

In [28]:
test1_pred = model.predict(test1_fea)
test2_pred = model.predict(test2_fea)
test3_pred = model.predict(test3_fea)

In [29]:
len(test1_pred)

300

In [31]:
# regression cross validation
rng = np.random.RandomState(123)
kf = KFold(n_splits=3, shuffle=True, random_state=rng)
print("在3折数据上的交叉验证")

for train_index, test_index in kf.split(train_fea):
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror',
                                 n_estimators=300, 
                                 max_depth=6,
                                 subsample=0.6,
                                 colsample_bytree=0.8,learning_rate=0.1,random_state=0)
    xgb_model.fit(train_fea[train_index],train_label[train_index])                             
    predictions = xgb_model.predict(train_fea[test_index])
    actuals = train_label[test_index]
    print("均方根误差:")
    print(np.sqrt(mean_squared_error(actuals, predictions)))
    print('拟合优度')
    print(xgb_model.score(train_fea[test_index],train_label[test_index]))

    
    
# Regression grid search for optimal hyperparameters
model = xgb.XGBRegressor(objective='reg:squarederror',
                         subsample=0.6, 
                         colsample_bytree=0.8, 
                         random_state=0,nthread=8)
param_dict = {'max_depth': [5,6,4,7],
              'n_estimators': [800,1000,1200],
              'learning_rate':[0.05,0.1,0.2]}
              
clf = GridSearchCV(model, param_dict, cv=10, verbose=1 , scoring='r2')
clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)


在3折数据上的交叉验证
均方根误差:
0.5514872325056268
拟合优度
0.48346335085517167
均方根误差:
0.5450997686007977
拟合优度
0.5227430332509759
均方根误差:
0.5196535871886602
拟合优度
0.547531596647326
Fitting 10 folds for each of 36 candidates, totalling 360 fits
0.547516598563219
{'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 800}


In [32]:
# predict
test3_pred = clf.predict(test3_fea)
ttest3 = pd.read_csv(os.path.join(label_path, 'test_set3.txt'), names=['file'])

test3_res = pd.DataFrame(ttest3['file'])
test3_res['mos'] = test3_pred

In [33]:
# Read baseline: test3 results of fea340+svr
import pandas as pd
import numpy as np
import os
path = r'./pred/baseline_feas340_cv10_grid'
path_test3 = pd.read_csv(os.path.join(path, 'DFGC-test3_DFGC1st_withstd_feats340_pred.txt'), names=['file','mos'])
path_test3

,file,mos
0,C1/3-1-2-submit-73479.mp4,2.161089
1,C1/3-2-5-submit-73479.mp4,2.470070
2,C1/4-1-1-submit-73479.mp4,1.990804
3,C1/4-2-2-submit-73479.mp4,2.032979
4,C1/8-1-1-submit-73479.mp4,1.520394
...,...,...
115,C3/8-2-10-submit-00000.mp4,2.135617
116,C3/8-2-10-submit-92147.mp4,2.059752
117,C3/8-2-10-submit-92584.mp4,1.808838
118,C3/8-2-10-submit-93014.mp4,2.206212


In [34]:
# Average the two test3 and save the file
test3_res['mos2'] = path_test3['mos']
temp3 = test3_res[['mos', 'mos2']]
test3_res["avg"] = temp3.mean(axis=1)
test3_res = test3_res.drop(['mos','mos2'],axis = 1)
test3_res.to_csv(os.path.join(r'./pred/combine_test3/', 'Test3_preds.txt'), index=None, header=None)